# Tests of reading in excel spreadsheet

This has been exported from google sheets. I am particularly interested in being able to get hold of the notes

In [1]:
import pandas as pd
from pathlib import Path

In [2]:
datapath = Path.cwd().parent.parent / "data" / "spec1d"

In [3]:
df = pd.read_excel(datapath / "All-Lines-MUSE-NGC-346.xlsx")

In [4]:
df

,Index,Region,λ(obs),λ(HM),Type,ID,Strength,Cont method,Comments
0,4.000000,GLOB-E,4604.944824,4601.29,NaN,NaN,NaN,NaN,Test of formatting of comments field. I have s...
1,7.000000,GLOB-E,4608.694824,4607.25,NaN,NaN,NaN,NaN,NaN
2,11.000000,GLOB-E,4613.694824,NaN,NaN,NaN,NaN,NaN,NaN
3,14.000000,GLOB-E,4617.444824,NaN,NaN,NaN,NaN,NaN,NaN
4,22.000000,GLOB-E,4627.444824,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
463,3779.000000,GLOB-E,9323.694824,NaN,Telluric?,NaN,NaN,NaN,NaN
464,3785.000000,GLOB-E,9331.194824,NaN,Telluric?,NaN,NaN,NaN,NaN
465,3790.000000,GLOB-E,9337.444824,NaN,Telluric?,NaN,NaN,NaN,NaN
466,3817.005644,GLOB-E,9371.201879,NaN,?,NaN,NaN,NaN,NaN


So, that gave me the table, but I do not see any of the notes.

In [62]:
set(str(x).rstrip('?') for x in df.Type)

{'',
 '****',
 'Deep Neutral',
 'Fe',
 'Fe, Ni',
 'H I, Ni',
 'High Neb',
 'High Perm',
 'Low Neb',
 'Low Perm',
 'Med Neb',
 'Med Perm',
 'Neutral',
 'Noise',
 'Sky',
 'Telluric',
 'nan'}

In [5]:
import openpyxl

In [23]:
workbook = openpyxl.load_workbook(datapath / "All-Lines-MUSE-NGC-346.xlsx", data_only=True)
sheet = workbook.active
sheet

<Worksheet "Master line list">

In [27]:
pd.DataFrame(list(sheet.values)[1:], columns=list(sheet.values)[0])


,Index,Region,λ(obs),λ(HM),Type,ID,Strength,Cont method,Comments,None,...,None,None,None,None,None,None,None,None,None,None
0,4.0,GLOB-E,4604.944824,4601.29,None,None,NaN,None,Test of formatting of comments field. I have s...,None,...,None,None,None,None,None,None,None,None,None,None
1,7.0,GLOB-E,4608.694824,4607.25,None,None,NaN,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,11.0,GLOB-E,4613.694824,NaN,None,None,NaN,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,14.0,GLOB-E,4617.444824,NaN,None,None,NaN,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,22.0,GLOB-E,4627.444824,NaN,None,None,NaN,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,NaN,None,NaN,NaN,None,None,NaN,None,None,None,...,None,None,None,None,None,None,None,None,None,None
998,NaN,None,NaN,NaN,None,None,NaN,None,None,None,...,None,None,None,None,None,None,None,None,None,None
999,NaN,None,NaN,NaN,None,None,NaN,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1000,NaN,None,NaN,NaN,None,None,NaN,None,None,None,...,None,None,None,None,None,None,None,None,None,None


DimensionHolder(<bound method Worksheet._add_row of <Worksheet "Master line list">>,
                {})

In [9]:
cell = sheet['A1']

In [10]:
cell.comment.text, cell.comment.author

('Index in the wavelength array of the line peak. Note that this is 0-based, add 1 to get the index in FITS convention',
 'None')

Aha, so it turns out that the notes are called "Comments" in the excel file. OK, that is easy then.

In [56]:
pd.DataFrame(
    [
        [
            # Take content from non-empty comments
            x.comment.content if x.comment else ""
            for x in row
            # Only use the columns we want
            if x.column_letter in "ABCDEFGHI"
        ]
        for row in sheet.rows
        # And only use rows that have at least some data
        if any(x.value for x in row)
    ]
)

,0,1,2,3,4,5,6,7,8
0,Index in the wavelength array of the line peak...,Which DS9 region was used to identify this li...,Observed wavelength of center of peak pixel. ...,Wavelength in system frame of midpoint between...,Broad class of line\n\nOne of:\nSky\nHigh Neb:...,Identification of line with rest wavelength in...,Some approximate measure of the line intensity...,Which type (or types) of continuum subtraction...,
1,,,,,,,,,
2,,,,,,,,,
3,,,,,,,,,
4,,,,,,,,,
...,...,...,...,...,...,...,...,...,...
464,,,,,,,,,
465,,,,,,,,,
466,,,,,,,,,
467,Where did these last two rows come from?\n\t-W...,,,,,,,,


In [58]:
pd.read_excel??

In [51]:
any(x.value for x in list(sheet.rows)[468])

True

So, the above works for automatically extracting all the notes in the same shape as the original table

What happens when it is a Sheets "comment" rather than "note"?

In [22]:
sheet['A468'].comment


Comment: Where did these last two rows come from?
	-William Henney by None

The author is included in the content field, while the author field is still `None`

What happens when we have several comments on the same cell?

In [18]:
x = sheet['F455'].comment
x

Comment: Looks like maybe another Ca I?
	-William Henney
Esteban+ 2004 have Fe I] 9218.47, which is a possibility
	-William Henney by None

In [20]:
x.content

'Looks like maybe another Ca I?\n\t-William Henney\nEsteban+ 2004 have Fe I] 9218.47, which is a possibility\n\t-William Henney'

They all get concatenated into the `.content` field